In [1]:
!pip install wfdb
!pip install biosppy
!pip install opencv-python

In [2]:
from __future__ import division, print_function

import json
import sys
import os
from glob import glob
import re
import wfdb
import cv2
import pandas as pd
import numpy as np
import biosppy
import matplotlib.pyplot as plt
from sklearn.utils import class_weight

from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions
from tensorflow.keras.models import save_model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import History
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, ELU, Dropout, Dense, Flatten
from tensorflow.keras import backend as K
import tensorflow as tf

In [3]:
def signal_to_image(signal, folder_name, record_ind, signal_ind):
    fig = plt.figure(frameon=False)
    plt.plot(signal, linewidth=3.5) 
    plt.xticks([]), plt.yticks([])
    for spine in plt.gca().spines.values():
        spine.set_visible(False)

    filename = folder_name + '/' + str(record_ind) + '_' + str(signal_ind) + '.png'
    
    fig.savefig(filename)
    im_gray = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    im_gray = cv2.resize(im_gray, (128, 128))
    cv2.imwrite(filename, im_gray)
    plt.close(fig)

    return im_gray

In [4]:
def create_pic(beats, symbols, signals, record_ind, signal_ind, file, directory):  
    class_to_idx = {'nor': 1, 'lbb': 2, 'rbb': 5, 'apc': 0, 'pvc': 4, 'pab': 3, 'veb': 6, 'vfw': 7}
    idx_to_class = dict(zip(class_to_idx.values(), class_to_idx.keys()))

    symbol_to_label = {'N':'nor', 'L':'lbb', 'R':'rbb', 'A':'apc', 'V':'pvc', '/':'pab', 'E':'veb', '!':'vfw'}  
    for i in range(len(beats)):
        if symbols[i] in list(symbol_to_label.keys()):
            left_ind = 0 if i == 0 else beats[i - 1] + 20
            right_ind = len(signals) if i == len(beats) - 1 else beats[i + 1] - 20
            signal = signals[left_ind: right_ind]

            signal_to_image(signal, directory, record_ind, signal_ind)
    
            with open(file, 'a') as f:
                f.write(str(record_ind) + '_' + str(signal_ind) + ' ' + str(class_to_idx[symbol_to_label[symbols[i]]]))
                f.write('\n')
                
            signal_ind += 1

In [5]:
records = ['101', '105', '114', '118', '124', '201', '210', '217']
for record_ind, record in enumerate(records):
    print("Current index: " + str(record_ind))
    signals = wfdb.rdsamp("test/" + record, channels=[0])[0]
    ann = wfdb.rdann("test/" + record, 'atr') 
    symbols = ann.symbol
    beats = list(ann.sample)

    create_pic(beats, symbols, signals, record_ind, 0, "labels-test.txt", 'test-imgs')

Current index: 0
Current index: 1
Current index: 2
Current index: 3
Current index: 4
Current index: 5
Current index: 6
Current index: 7


In [ ]:
records = ['100', '102', '103', '104', '106', '107', '108', '109','111', '112','113','115','116','117','119','121','122','123','200','202','203','205','207','208','209','212','213','214','215','219','220','221','222','223','228','230','231','232','233','234']
for record_ind, record in enumerate(records):
    print("Current index: " + str(record_ind))
    signals = wfdb.rdsamp("train/" + record, channels=[0])[0]
    ann = wfdb.rdann("train/" + record, 'atr') 
    symbols = ann.symbol
    beats = list(ann.sample)

    create_pic(beats, symbols, signals, record_ind, 0, "labels-train.txt", 'train-imgs')

Current index: 0
